In [1]:
# for Python 2: use print only as a function
# from __future__ import print_function

In [2]:
import ast
import gensim

import pandas as pd
import numpy as np
from collections import defaultdict
import collections 

In [3]:
path = 'C:\Users\Pushpendra\Desktop\qa\datascience_datasets_for_doc_similarity.csv'

In [4]:
d_set = pd.read_csv(path,header=None,names=['tag','title','body'],dtype=object)

In [5]:
print gensim.utils.simple_preprocess(d_set['body'][0])   
d_set.head()

[u'adding', u'more', u'data', u'does', u'not', u'always', u'help', u'however', u'you', u'can', u'get', u'an', u'estimate', u'if', u'more', u'data', u'will', u'help', u'you', u'by', u'the', u'following', u'procedure', u'make', u'plot', u'on', u'the', u'axis', u'is', u'the', u'amount', u'of', u'training', u'examples', u'starting', u'at', u'one', u'example', u'per', u'class', u'going', u'to', u'wherever', u'you', u'are', u'currently', u'the', u'axis', u'shows', u'the', u'error', u'now', u'you', u'should', u'add', u'two', u'curves', u'training', u'and', u'test', u'error', u'for', u'low', u'the', u'training', u'error', u'should', u'be', u'very', u'low', u'almost', u'and', u'the', u'test', u'error', u'very', u'high', u'with', u'enough', u'data', u'they', u'should', u'be', u'about', u'the', u'same', u'by', u'plotting', u'those', u'curves', u'you', u'can', u'make', u'an', u'educated', u'guess', u'how', u'much', u'more', u'data', u'will', u'give', u'you', u'how', u'much', u'improvement', u'bloc

,tag,title,body
0,"['machine-learning', 'classification', 'evalua...",How to improve an existing (trained) classifier?,<p>Adding more data does not always help. Howe...
1,"['machine-learning', 'r', 'logistic-regression...","Random Forest, Type - Regression, Calculation ...",<p>MSE is measure of error of the overall regr...
2,['visualization'],How to analyze which site has most numbers,<p>The most obvious way of visualizing this is...
3,['bigdata'],Privacy through fake data?,<p>There's an inherent problem that anybody wh...
4,"['r', 'data-wrangling']",When to choose character instead of factor in R?,<p>Factors are stored as numbers and a table o...


## Building corpus for Doc2Vec model

In [6]:
body_corpus = []
for i, line in enumerate(d_set['body']): 
    body_corpus.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]))   
print (body_corpus)[:2], '\n'    

title_corpus = []
for i, line in enumerate(d_set['title']):
    title_corpus.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]))
print (title_corpus)[:2]

[TaggedDocument(words=[u'adding', u'more', u'data', u'does', u'not', u'always', u'help', u'however', u'you', u'can', u'get', u'an', u'estimate', u'if', u'more', u'data', u'will', u'help', u'you', u'by', u'the', u'following', u'procedure', u'make', u'plot', u'on', u'the', u'axis', u'is', u'the', u'amount', u'of', u'training', u'examples', u'starting', u'at', u'one', u'example', u'per', u'class', u'going', u'to', u'wherever', u'you', u'are', u'currently', u'the', u'axis', u'shows', u'the', u'error', u'now', u'you', u'should', u'add', u'two', u'curves', u'training', u'and', u'test', u'error', u'for', u'low', u'the', u'training', u'error', u'should', u'be', u'very', u'low', u'almost', u'and', u'the', u'test', u'error', u'very', u'high', u'with', u'enough', u'data', u'they', u'should', u'be', u'about', u'the', u'same', u'by', u'plotting', u'those', u'curves', u'you', u'can', u'make', u'an', u'educated', u'guess', u'how', u'much', u'more', u'data', u'will', u'give', u'you', u'how', u'much', 

In [7]:
model_for_body = gensim.models.doc2vec.Doc2Vec(size=30, min_count=2, iter=50)
model_for_title = gensim.models.doc2vec.Doc2Vec(size=15, min_count=2, iter=100)

In [8]:
model_for_body.build_vocab(body_corpus)
model_for_title.build_vocab(title_corpus)

In [9]:
print len(model_for_body.vocab)
print len(model_for_title.vocab)

7330
1014


In [10]:
print model_for_body.vocab['regression'].count
print model_for_title.vocab['regression'].count

231
62


## training Doc2Vec model

In [11]:
%time model_for_body.train(body_corpus, total_examples=model_for_body.corpus_count) 
%time model_for_title.train(title_corpus, total_examples=model_for_title.corpus_count) 

Wall time: 36.2 s
Wall time: 4.35 s


984400

## Taking user's query and processing them

In [12]:
user_query = "why is overfitting bad in machine learning ?"  

user_body_vec = model_for_body.infer_vector(gensim.utils.simple_preprocess(user_query))
user_title_vec = model_for_title.infer_vector(gensim.utils.simple_preprocess(user_query))

print user_body_vec, '\n'
print user_title_vec

[ 0.03127864  0.02093349  0.03002832  0.00866496 -0.00605498  0.01654723
  0.00773027 -0.0001666  -0.00406452  0.06625747  0.03237529  0.02781813
  0.04272798 -0.01236193 -0.0083826   0.00299702 -0.07634215 -0.0488191
 -0.03117685 -0.03597934 -0.00298677  0.00257462  0.01823049 -0.02434584
  0.01248353 -0.03326915 -0.01310654  0.04946647 -0.01679867  0.00603149] 

[ 0.09981605  0.12393562 -0.06997055 -0.07479089  0.01536176 -0.04371031
 -0.00609764 -0.06227994  0.00910525  0.0320807   0.17170812  0.10681643
  0.27942887  0.15855436  0.03077954]


## Retrieving top 5, most relevant documents corresponding to any user' s query

In [13]:
body_sims = model_for_body.docvecs.most_similar([user_body_vec],topn=5) 
print "body_sims: ", body_sims, '\n'  
# for i in body_sims:   
#     print (' '.join(body_corpus[i[0]].words)), '\n'    
    
    
title_sims = model_for_title.docvecs.most_similar([user_title_vec],topn=5) 
print "title_sims: ", title_sims, '\n'  
# for i in title_sims:   
#     print (' '.join(title_corpus[i[0]].words)), '\n'
print '\n\n'
    
for i in title_sims:
    print ' '.join(body_corpus[i[0]].words), '\n'      # infering body from most similar title
    

body_sims:  [(618, 0.5026001930236816), (1193, 0.4658726155757904), (411, 0.43993499875068665), (883, 0.43816816806793213), (78, 0.41403472423553467)] 

title_sims:  [(1011, 0.8406146168708801), (356, 0.7910738587379456), (836, 0.6973468661308289), (534, 0.6693466305732727), (229, 0.6513209342956543)] 




overfitting is em empirically em bad suppose you have data set which you split in two test and training an overfitted model is one that performs much worse on the test dataset than on training dataset it is often observed that models like that also in general perform worse on additional new test datasets than models which are not overfitted one way to understand that intuitively is that model may use some relevant parts of the data signal and some irrelevant parts noise an overfitted model uses more of the noise which increases its performance in the case of known noise training data and decreases its performance in the case of novel noise test data the difference in performance betw

## Saving Document retrieval model on Disk

In [14]:
# store the model to mmap-able files
model_for_title.save('my_model_for_title.doc2vec')

# load the model back
model_for_title_loaded = gensim.models.doc2vec.Doc2Vec.load('my_model_for_title.doc2vec') 


In [15]:
model_for_title_loaded.docvecs.most_similar([user_title_vec],topn=5) 

[(1011, 0.8406146168708801),
 (356, 0.791073739528656),
 (836, 0.6973468661308289),
 (534, 0.6693466305732727),
 (229, 0.6513208746910095)]